In [1]:

pip install PyMuPDF transformers Flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 15.8 MB/s eta 0:00:00


In [12]:
!pip install PyMuPDF google-colab

import fitz  # PyMuPDF
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

# Replace with the actual path to your PDF file in Google Drive
pdf_path = '/content/drive/My Drive/Colab Notebooks/test1.pdf'
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)


Streaming output truncated to the last 5000 lines.
Court or from one High Court to another. 
PART E
PROVISIONS AS TO THE COMPTROLLER AND AUDITOR-GENERAL OF INDIA 
12. (1) There shall be paid to the Comptroller and Auditor-General of 
India a salary at the rate of *four thousand rupees per mensem.
(2) The person who was holding office immediately before the 
commencement of this Constitution as Auditor-General of India and has 
become on such commencement the Comptroller and Auditor-General of India 
under article 377 shall in addition to the salary specified in sub-paragraph (1) of 
this paragraph be entitled to receive as special pay an amount equivalent to the 
difference between the salary so specified and the salary which he was drawing 
as Auditor-General of India immediately before such commencement.
(3) The rights in respect of leave of absence and pension and the other 
conditions of service of the Comptroller and Auditor-General of India shall be 
governed or shall continue to

In [14]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token # using eos_token as pad_token

# Tokenizing the extracted text
tokenized_text = tokenizer(pdf_text, return_tensors="pt", padding=True, truncation=True, max_length=512)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, GPT2Tokenizer
from torch.utils.data import Dataset

# Assume 'tokenized_text' contains tokenized data with input IDs
class CustomDataset(Dataset):
    def __init__(self, encodings):
        # Create labels by copying input_ids; this will allow model to compute loss
        self.encodings = {key: val for key, val in encodings.items()}
        self.encodings['labels'] = self.encodings['input_ids'].clone()

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Add a padding token to the tokenizer - THIS WAS MISSING
tokenizer.pad_token = tokenizer.eos_token # using eos_token as pad_token

# Tokenize the text - passing the actual pdf_text variable instead of the string "pdf_text"
tokenized_text = tokenizer(pdf_text, return_tensors="pt", max_length=512, padding=True, truncation=True)

# Create the dataset
dataset = CustomDataset(tokenized_text)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=1,
    num_train_epochs=1,
    save_steps=10_000,
    logging_dir='./logs',
    report_to="none"  # Disable wandb integration
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step,Training Loss


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json')